# Introduction to Data Science 
# Lecture 24: Temporal data analysis and applications to stock analysis
*COMP 5360 / MATH 4100, University of Utah, http://datasciencecourse.net/*

In this lecture, we'll cover 
* temporal data analysis 
* importing stock data using the pandas_datareader python package 
* visualizing time series 
* rolling means
* time-domain vs frequency-domain analysis 

Parts of this lecture were adapted from a [previous lecture by *Curtis Miller*](https://github.com/datascience-course/2019-datascience-lectures/tree/master/08-time-series) 
and a [lecture on Time Series and Spectral Analysis by James Holland Jones](http://web.stanford.edu/class/earthsys214/notes/series.html). 

Further reading: 
+ Yves Hilpisch, Python for Finance, O'Reilly, (2014) [link](http://proquest.safaribooksonline.com.ezproxy.lib.utah.edu/book/programming/python/9781491945360).

For a more complete treatment, take Math 5075 (Time Series Analysis). 

## Temporal data analysis

A *time series* is a series of data points indexed by time, $x_i = x(t_i)$, for $i=1,\ldots,n$. Examples frequently occur in 
* weather forecasting, 
* mathematical finance (stocks),
* electricity demand in a power grid, 
* keystrokes on a computer, and 
* any applied science and engineering which involves temporal measurements

*Temporal data analysis* or *time series analysis* is just the study of such data. 

As a first example of time series data, we'll consider stocks and *mathematical finance*. 

##  Mathematical finance 

Prior to the 1980s, banking and finance were well-known for being "boring"; investment banking was distinct from commercial banking and the primary role of the industry was handling "simple" (at least in comparison to today) financial instruments, such as loans. Deregulation under the Regan administration, coupled with an influx of mathematics and computing power have transformed the industry from the "boring" business of banking to what it is today. 

* Advanced mathematics, such as analysis of the [Black-Scholes model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model), is now essential to finance. 
* Algorithms are now responsible for making split-second decisions. In fact, [the speed at which light travels is a limitation when designing trading systems](http://www.nature.com/news/physics-in-finance-trading-at-the-speed-of-light-1.16872). 
* [Machine learning and data mining techniques are popular](http://www.ft.com/cms/s/0/9278d1b6-1e02-11e6-b286-cddde55ca122.html#axzz4G8daZxcl) in the financial sector. For example, **high-frequency trading (HFT)** is a branch of algorithmic trading where computers make thousands of trades in short periods of time, engaging in complex strategies such as statistical arbitrage and market making. HFT was responsible for phenomena such as the [2010 flash crash](https://en.wikipedia.org/wiki/2010_Flash_Crash) and a [2013 flash crash](http://money.cnn.com/2013/04/24/investing/twitter-flash-crash/) prompted by a hacked [Associated Press tweet](http://money.cnn.com/2013/04/23/technology/security/ap-twitter-hacked/index.html?iid=EL) about an attack on the White House.

In [ ]:
#imports and setup

import numpy as np
import pandas as pd
import pandas_datareader
import yfinance as yf
from datetime import datetime
from scipy.signal import periodogram

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

## Getting and Visualizing Stock Data

As described [here](https://yfinance-python.org/reference/index.html), the yfinance package provides easy access to Yahoo! Finance’s API to retrieve market data. It includes classes and functions for downloading historical market data, accessing ticker information, managing cache, and more.

The following are the publicly available classes, and functions exposed by the yfinance package:

    Ticker: Class for accessing single ticker data.
    Tickers: Class for handling multiple tickers.
    Market: Class for accessing market summary.
    download: Function to download market data for multiple tickers.
    Search: Class for accessing search results.
    Sector: Domain class for accessing sector information.
    Industry: Domain class for accessing industry information.
    Market: Class for accessing market status & summary.
    EquityQuery: Class to build equity query filters.
    FundQuery: Class to build fund query filters.
    screen: Run equity/fund queries.
    enable_debug_mode: Function to enable debug mode for logging.
    set_tz_cache_location: Function to set the timezone cache location.

Note there are various sources for financial date (yahoo, fred, iex...), and pandas_datareader also has many ways to access this type of data. In this lecture we will primarily use the download function from yfinance.

Now let's get some data.

In [ ]:
start = datetime(2016, 1, 29)
end = datetime(2023, 1, 29)
ticker = 'AAPL'

AAPL = yf.download(ticker, start, end)
print(type(AAPL))
print(AAPL.tail())
print(AAPL.info())

What does this data mean? 
* **high** is the highest price of the stock on that trading day, 
* **low** the lowest price of the stock on that trading day, 
* **open** is the price of the stock at the beginning of the trading day (it need not be the closing price of the previous trading day)
* **close** the price of the stock at closing time
* **volume** indicates how many stocks were traded 

### Visualizing Stock Data

Now that we have stock data we can visualize it using the `matplotlib` package, called using a convenience method, `plot()` in pandas.

In [ ]:
AAPL["Close"].plot(grid = True);

### Plotting multiple stocks together

For a variety of reasons, we may wish to plot multiple financial instruments together including:
* we may want to compare stocks
* compare them to the market or other securities such as [exchange-traded funds (ETFs)](https://en.wikipedia.org/wiki/Exchange-traded_fund).

Here, we plot the adjusted close for several stocks together. 

In [ ]:
MSFT = yf.download('MSFT', start, end)
GOOG = yf.download('GOOG', start, end)
# Below I create a DataFrame consisting of the closing price of these stocks, first by making a list of these objects and using the join method
close = pd.DataFrame({ "AAPL": AAPL['Close', 'AAPL'], "MSFT": MSFT['Close', 'MSFT'], "GOOG": GOOG['Close', 'GOOG']})
close.tail()

In [ ]:
close.plot(grid = True);

**Q:** Why is this plot difficult to read?

It plots the *absolute price* of stocks with time. While absolute price is important, frequently we are more concerned about the *relative change* of an asset rather than its absolute price. Also, Microsoft stock is more expensive than Apple or Google stock, and this difference makes Apple and Google stock appear less volatile than they truly are (that is, their price appears not to vary as much with time).

One solution is to use two different scales when plotting the data; one scale will be used by Apple and Microsoft stocks, and the other by Google.

In [ ]:
close.plot(secondary_y = ["AAPL", "MSFT"], grid = True);

But, this solution clearly has limitations. We only have two sides of the plot to add more labels! 

A "better" solution is to plot the information we actually want. One option is to plot the *stock returns since the beginning of the period of interest*:

$$
\text{return}_{t,0} = \frac{\text{price}_t}{\text{price}_0}
$$

This requires transforming the data, which we do using a *lambda function*. 

In [ ]:
# df.apply(arg) will apply the function arg to each column in df, and return a DataFrame with the result
# Recall that lambda x is an anonymous function accepting parameter x; in this case, x will be a pandas Series object
stock_return = close.apply(lambda x: x / x[0])
stock_return.tail()

In [ ]:
stock_return.plot(grid = True).axhline(y = 1, color = "black", lw = 1);

This is a much more useful plot! Note: 
* We can now see how profitable each stock was since the beginning of the period. 
* Furthermore, we see that these stocks are highly correlated; they generally move in the same direction, a fact that was difficult to see in the other charts.

Alternatively, we could plot the change of each stock per day. One way to do so would be to use the *percentage increase of a stock*:
$$
\text{increase}_t = \frac{\text{price}_{t} - \text{price}_{t-1}}{\text{price}_t}
$$

or the *log difference*.

$$
\text{change}_t = \log\left( \frac{\text{price}_{t}}{\text{price}_{t - 1}} \right) = \log(\text{price}_{t}) - \log(\text{price}_{t - 1})
$$

Here, $\log$ is the natural log. Log difference has a desirable property: the sum of the log differences can be interpreted as the log total change (as a percentage) over the period summed. Log differences also more cleanly correspond to how stock prices are modeled in continuous time.

We can obtain and plot the log differences of the data as follows.

In [ ]:
stock_change = close.apply(lambda x: np.log(x) - np.log(x.shift(1))) # shift moves dates back by 1.
stock_change.head()

In [ ]:
stock_change.plot(grid = True).axhline(y = 0, color = "black", lw = 1);

Do you prefer to plot stock return or log difference? 

* Looking at returns since the beginning of the period make the overall trend of the securities apparent. 
* Log difference, however, emphasizes changes between days. 

### Comparing stocks to the overall market 

We often want to compare the performance of stocks to the performance of the overall market. 
[SPY](https://finance.yahoo.com/quote/SPY/) is the ticker symbol for the SPDR S&P 500 exchange-traded mutual fund (ETF), which is a fund that has roughly the stocks in the [S&P 500 stock index](https://finance.yahoo.com/quote/%5EGSPC?p=^GSPC). 
This serves as one measure for the overal market.

In [ ]:
SPY = yf.download('SPY', start, end)
SPY.tail()

In [ ]:
close['SPY'] = SPY['Close','SPY']
close.head()

In [ ]:
stock_return['SPY'] = close[['SPY']].apply(lambda x: x / x[0])
stock_return.head()

In [ ]:
stock_return.plot(grid = True).axhline(y = 1, color = "black", lw = 1);

In [ ]:
stock_change['SPY'] = close[['SPY']].apply(lambda x: np.log(x) - np.log(x.shift(1))) # shift moves dates back by 1.
stock_change.head()

In [ ]:
stock_change.plot(grid = True).axhline(y = 0, color = "black", lw = 1);

### Moving Averages

For a time series $x_t$, the *$q$-day moving average at time $t$*, denoted $MA^q_t$, is the average of $x_t$ over the past $q$ days, 
$$
MA^q_t = \frac{1}{q} \sum_{i = 0}^{q-1} x_{t - i}
$$

The [`rolling`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) function in Pandas provides functionality for computing moving averages. We'll use it to create a 20-day moving average for Apple stock data and plot it alongside the stock price. 

In [ ]:
AAPL["20d","AAPL"] = AAPL["Close","AAPL"].rolling(window = 20, center = False).mean()
AAPL.head(30)

Notice how late the rolling average begins. It cannot be computed until twenty days have passed. Note that this becomes more severe for slower moving averages. 

In [ ]:
AAPL[["Close", "20d"]].tail(300).plot(grid = True); 

Notice that the moving averages "smooths" the time series. This can sometimes make it easier to identify trends. The larger $q$, the less responsive a moving average is to fast fluctuations in the series $x_t$. 
So, if these fast fluctuations are considered "noise", a moving average will identify the "signal". 
* *Fast moving averages* have smaller $q$ and more closely follow the time series. 
* *Slow moving averages* have larger $q$ and respond less to the fluctuations of the stock.

Let's compare the 20-day, 50-day, and 200-day moving averages.


In [ ]:
AAPL["50d","AAPL"] = AAPL["Close","AAPL"].rolling(window = 50, center = False).mean()
AAPL["200d","AAPL"] = AAPL["Close","AAPL"].rolling(window = 200, center = False).mean()

AAPL[["Close", "20d", "50d", "200d"]].tail(500).plot(grid = True); 

The 20-day moving average is the most sensitive to fluctuations, while the 200-day moving average is the least sensitive. 

### Trading strategies and backtesting 


**Trading** is the practice of buying and selling financial assets for the purpose of making a profit. Traders develop **trading strategies** that a computer can use to make trades. Sometimes, these can be very complicated, but other times traders make decisions based on finding patterns or trends in charts. 

One example is called the [moving average crossover strategy](http://www.investopedia.com/university/movingaverage/movingaverages4.asp). 
This strategy is based on two moving averages, a "fast" one and a "slow" one. The strategy is:

* Trade the asset when the fast moving average crosses over the slow moving average.
* Exit the trade when the fast moving average crosses over the slow moving average again.

A trade will be prompted when the fast moving average crosses from below to above the slow moving average, and the trade will be exited when the fast moving average crosses below the slow moving average later.

This is the outline of a complete strategy and we already have the tools to get a computer to automatically implement the strategy.

But before we decide if we want to use it, we should first evaluate the quality of the strategy. The usual means for doing this is called **backtesting**, which is looking at how profitable the strategy is on historical data. 

You could now write python code that could implement and backtest a trading strategy. There are also lots of python packages for this:  
* [**pyfolio**](https://quantopian.github.io/pyfolio/) (for analytics)
* [**zipline**](http://www.zipline.io/beginner-tutorial.html) (for backtesting and algorithmic trading), and 
* [**backtrader**](https://www.backtrader.com/) (also for backtesting and trading). 




# Time-domain vs frequency-domain analysis  

So far, we have thought about a time series $x(t)$ in the "time domain". But, for some time series, it is easier to describe them in terms of the "frequency domain".

For example, a good way to describe the function  
$$
x(t) = \cos\left( 2 \pi f t \right)
$$
is as an oscillating function with frequency $f$ (or period $1/f$). 

According to [Fourier analysis](https://en.wikipedia.org/wiki/Fourier_transform), we can decompose any signal into its frequency components, 
$$
x(t) = \sum_{n=-\infty}^\infty \hat{x}(n) \ e^{2 \pi i n t}
\qquad \qquad t \in [0,1] 
$$
or 
$$
x(t) = \int_{-\infty}^\infty \hat{x}(f) \ e^{2 \pi i f t} \ df
\qquad \qquad t \in [-\infty,\infty]. 
$$

The 
[*power spectral density* or *periodogram*](https://en.wikipedia.org/wiki/Spectral_density) 
$S_{xx}(f) \approx |\hat x(f) |^2$ 
of a time series $x(t)$ describes the distribution of power into the frequency components that compose that signal. 

There are lots of time-dependent signals that are periodic or at least some of the signal is periodic. Examples: 
* [sunspots](https://en.wikipedia.org/wiki/Sunspot) follow an 11 year cycle. So if $x(t)$ was a time series representing the "strength" of the sunspot, we would have that $|\hat{x}(f)|^2$ would be large at $f = 1/11$. (Remember period = 1/frequency.)  
* The temperature in SLC. Here, we can decompose the temperature into a part that is varying over the course of a year, the part that varies over the day, and the "remaining" part.
* $\ldots$


We can compute the power spectral density using the scipy function 
[`periodogram`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.periodogram.html). 

To illustrate this, we'll follow the 
[course notes of James Holland Jones on Time Series and Spectral Analysis](http://web.stanford.edu/class/earthsys214/notes/series.html) 
and consider 
[historical measles data from New York City posted by Ben Bolker](https://ms.mcmaster.ca/~bolker/measdata.html). 


### Measles data

We can download the monthly measles data from New York City between 1928 and 1964. 

In [ ]:
df = pd.read_csv("nycmeas.dat", sep=" ", names=["date","cases"])
df.head(20)

In [ ]:
df["cases"].plot(grid = True); 
plt.show()
plt.plot(df["date"].tolist(), df["cases"].tolist())
plt.show()

Looking at the plot, we observe that the series is very regular with "periodically occuring" spikes. It appears that approximately once a year, there is a significant measles outbreak. By computing the power spectrum, we can see which frequencies make up this time series. 

In [ ]:
cases = df["cases"].values
f, Pxx_den = periodogram(cases, fs=12, window="hamming")

plt.plot(f, Pxx_den);

Since there are 12 months per year, we set the measurement frequency argument in `periodogram` as fs=12. 

Clearly, the dominant frequency in this signal is 1 year. Why? 


**Q:** Is it useful to look at the power spectrum of stock data? 

**Exercise:** try it for the Apple stock data over the previous 10 years. 

In [ ]:
f, Pxx_den = periodogram(AAPL['Close', 'AAPL'].values, fs=365, window="hamming")

plt.plot(f[:20], Pxx_den[:20]);